In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from fastcore.test import test, operator

In [3]:
from pyproteonet.simulation.missing_values import simulate_mnars_thresholding, simulate_mcars
from pyproteonet.visualization import plot_hist
from pyproteonet.simulation.sampling import draw_normal_log_space
from pyproteonet.processing.aggregation import neighbor_sum
from pyproteonet.processing.dataset_transforms import normalize, logarithmize
from pyproteonet.processing.masking import train_test_non_missing_no_overlap_iterable
from pyproteonet.predictors.gnn import GnnPredictor
from pyproteonet.dgl.gnn_architectures import GAT
from pyproteonet.lightning.console_logger import ConsoleLogger

# Load Real World Dataset as Template

In [4]:
from test_utils import load_maxlfq_benchmark

In [5]:
maxlfq_benchmark = load_maxlfq_benchmark()

In [6]:
log_mu, log_sigma = 0.05647178595714227, 2.519063763272205

# Simulate Simple Data without any Errors

In [8]:
ds = draw_normal_log_space(molecule_set=maxlfq_benchmark.molecule_set,
                           log_mu=log_mu,
                           log_sigma=log_sigma,
                           samples=len(maxlfq_benchmark.samples),
                           molecule='protein_group', column='abundance_gt')
neighbor_sum(ds, molecule='protein_group', column='abundance_gt', mapping='protein_group-peptide',
             result_molecule='peptide', result_column='abundance', only_unique=False, inplace=True)

In [17]:
ds_gnn = normalize(logarithmize(ds))
train_mds, test_mds = train_test_non_missing_no_overlap_iterable(dataset=ds_gnn, train_frac=0.1, test_frac=0.2, molecule='peptide',
                                                                 non_missing_column='abundance')
logger = ConsoleLogger()
model = GAT(in_dim=3, hidden_dim=40, out_dim=1, num_heads=20)
gnn_predictor = GnnPredictor(mapping='protein_group-peptide', value_columns=['abundance'], molecule_columns=[], target_column='abundance',
                             model = model,
                             bidirectional_graph = True, missing_substitute_value=-3.0,
                             logger=logger, 
                            )
gnn_predictor.fit(train_mds=train_mds, test_mds=test_mds, max_epochs=20)
test(logger.logs['validation_r2'][-1], 0.9, operator.gt)

/hpi/fs00/home/tobias.pietz/mambaforge/envs/proteomics/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /hpi/fs00/home/tobias.pietz/mambaforge/envs/proteomi ...
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type | Params
--------------------------------
0 | _model | GAT  | 326 K 
--------------------------------
326 K     Trainable params
0         Non-trainable params
326 K     Total params
1.306     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/hpi/fs00/home/tobias.pietz/mambaforge/envs/proteomics/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/hpi/fs00/home/tobias.pietz/mambaforge/envs/proteomics/lib/python3.11/site-packages/lightning/pytorch/utilities/data.py:120: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
/hpi/fs00/home/tobias.pietz/mambaforge/envs/proteomics/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train

Training: 0it [00:00, ?it/s]

step5: train_loss:0.849453866481781 || train_mse:0.849453866481781 || train_rmse:0.9216582179069519 || train_mae:0.7396981120109558 || train_r2:0.5482026934623718 || train_pearson:0.7404071092605591 || epoch:0 || 


...

Validation: 0it [00:00, ?it/s]

step113: validation_loss:0.09648826718330383 || validation_mse:0.09648826718330383 || validation_rmse:0.3106255829334259 || validation_mae:0.15272946655750275 || validation_r2:0.9069094061851501 || validation_pearson:0.9523178935050964 || epoch:18 || 
step119: train_loss:0.10870572179555893 || train_mse:0.10870572179555893 || train_rmse:0.32970550656318665 || train_mae:0.1541898101568222 || train_r2:0.8898435831069946 || train_pearson:0.9433152079582214 || epoch:19 || 


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


step119: validation_loss:0.09142621606588364 || validation_mse:0.09142621606588364 || validation_rmse:0.3023676872253418 || validation_mae:0.14517317712306976 || validation_r2:0.9110947251319885 || validation_pearson:0.9545127749443054 || epoch:19 || 
